<a href="https://colab.research.google.com/github/Ryan-RE-Wang/BCI_project/blob/main/Download_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import seed
from collections import Counter
from datetime import datetime
import time
import os, sys

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, cohen_kappa_score, accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
sys.path.append('/content/drive/Shareddrives/BCI/sleep_study/sleep_study')
import sleep_study as ss

# Useful Functions

In [ ]:
import subprocess

def runcmd(cmd, verbose = False, *args, **kwargs):

    process = subprocess.Popen(
        cmd,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        text = True,
        shell = True
    )
    std_out, std_err = process.communicate()
    if verbose:
        print(std_out.strip(), std_err)
    pass

# Download file

In [ ]:
ss.init('/content/drive/Shareddrives/BCI/BCI_Data')
print('total number of sleep study files available:', len(ss.data.study_list))

age information stored in /content/age_file.csv
total number of sleep study files available: 0


In [ ]:
age_groups = list(zip(range(0, 18), range(1, 19))) + [(18, 100)]

tmp = np.load('/content/drive/Shareddrives/BCI/sleep_study/sleep_study/examples/wavelet/study_lists.npz', allow_pickle=True) 
study_lists = tmp["study_lists"] # filenames that are in each age group

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/Shareddrives/BCI/BCI_Data/Health_Data/SLEEP_STUDY.csv')
df.head()

,STUDY_PAT_ID,SLEEP_STUDY_ID,SLEEP_STUDY_START_DATETIME,SLEEP_STUDY_DURATION_DATETIME,AGE_AT_SLEEP_STUDY_DAYS
0,1,4789,2019-04-02 20:22:49,9:35:32,3338
1,7,12595,2018-12-05 18:20:31,11:22:32,775
2,10,22339,2018-01-07 18:41:16,11:14:13,4782
3,16,24241,2019-02-05 20:17:31,10:03:36,4408
4,22,23233,2019-07-29 18:50:11,11:15:36,1730


In [ ]:
df_diagnosis = pd.read_csv('/content/drive/Shareddrives/BCI/DIAGNOSIS.csv', index_col=0)
df_diagnosis.head()

,STUDY_DX_ID,STUDY_ENC_ID,STUDY_PAT_ID,DX_START_DATETIME,DX_END_DATETIME,DX_SOURCE_TYPE,DX_ENC_TYPE,DX_CODE_TYPE,DX_CODE,DX_NAME,DX_ALT_CODE,CLASS_OF_PROBLEM,CHRONIC_YN,PROV_ID
0,10001,55902925,13210,2018-09-07 00:00:00,2018-09-07 00:00:00,Primary Encounter Dx,Encounter Dx,ICD10,R56.9,Seizure,780.39,NaN,NaN,3001267.0
1,10017,55772269,12547,2014-11-01 00:00:00,2014-11-01 00:00:00,Secondary Encounter Dx,Encounter Dx,ICD9,790.7,Positive blood culture,R78.81,NaN,NaN,3001267.0
2,10018,55772269,12547,2014-11-01 19:58:00,2014-11-01 23:08:00,Admit Dx,Admit Dx,ICD9,796.4,Other abnormal clinical finding,R68.89,NaN,NaN,3001267.0
3,10019,55772269,12547,2014-11-01 19:58:00,2014-11-01 23:08:00,Final Dx,Final Dx,ICD9,796.4,Other abnormal clinical finding,R68.89,NaN,NaN,3001267.0
4,10020,56359534,7138,2019-05-18 00:00:00,2019-05-18 00:00:00,Secondary Encounter Dx,Encounter Dx,ICD10,J06.9,Viral URI,465.9,NaN,NaN,3001267.0


In [ ]:
# this can be parallelized to speed up the process
# for i, study_list in enumerate(study_lists):
for j, name in enumerate(study_lists[1]):
    if (j < 5):
        continue

    cmd = 'wget -r -N -c -np --user ryanwang --password ryan57231211 -A "{k}.*" https://physionet.org/files/nch-sleep/3.1.0/Sleep_Data/ -P /content/'.format(k=name)
    runcmd(cmd)

    a = df.loc[(df['STUDY_PAT_ID']==int(name.split('_')[0])) & (df['SLEEP_STUDY_ID']==int(name.split('_')[1])), 'SLEEP_STUDY_START_DATETIME'].values[0]

    features, _ = ss.data.get_demo_wavelet_features_and_labels(name)
    features = np.array(features)
    labels =  np.unique(df_diagnosis.loc[(df_diagnosis['STUDY_PAT_ID']==int(name.split('_')[0])) & (df_diagnosis['DX_START_DATETIME']<=a) & (df_diagnosis['DX_END_DATETIME']>=a), 'DX_CODE'].values)

    cmd = 'rm -r /content/physionet.org/'
    runcmd(cmd)

    print( )
    print(features.shape, labels.shape)
    
    fn = '/content/drive/Shareddrives/BCI/BCI_Data/Sleep_Data/wavelet_features' + name
    
    np.savez(fn, labels=labels, features=features)
    print('features from', name, 'i:{}, j:{}'.format(i, j))
    print(' ')


(850, 7, 12) (1,)
features from 541_24154 i:1, j:5
 

(994, 7, 12) (3,)
features from 556_6685 i:1, j:6
 

(891, 7, 12) (1,)
features from 646_23959 i:1, j:7
 

(1049, 7, 12) (2,)
features from 670_4933 i:1, j:8
 

(1015, 7, 12) (2,)
features from 889_22 i:1, j:9
 

(921, 7, 12) (1,)
features from 997_17593 i:1, j:10
 

(1072, 7, 12) (4,)
features from 1093_1408 i:1, j:11
 

(1029, 7, 12) (1,)
features from 1105_6529 i:1, j:12
 

(924, 7, 12) (6,)
features from 1216_23932 i:1, j:13
 

(831, 7, 12) (1,)
features from 1360_5413 i:1, j:14
 

(939, 7, 12) (1,)
features from 1360_6232 i:1, j:15
 

(975, 7, 12) (1,)
features from 1393_19294 i:1, j:16
 

(840, 7, 12) (1,)
features from 1468_8299 i:1, j:17
 

(884, 7, 12) (1,)
features from 1483_7963 i:1, j:18
 

(1061, 7, 12) (1,)
features from 1555_406 i:1, j:19
 

(1068, 7, 12) (2,)
features from 1630_25783 i:1, j:20
 

(888, 7, 12) (2,)
features from 1714_5065 i:1, j:21
 

(957, 7, 12) (3,)
features from 1945_25306 i:1, j:22
 

(978, 7, 1